In [1]:

'''
疑惑：alpha2_new=alpha2+ y2*(E1-E2)/n
这样的话，评估ａｃｃ为0.88
如果是：alpha2_new=alpha2+ y2*(E２-E１)/n
就是0.96，不知道为什么
alpha2_newunc=self.alphas[j]+self.Y[j]*(self.E[j]-self.E[i])/eta
'''

0.96


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from collections import Counter
import math


0.96


In [8]:
#load data
data=load_iris()
# print(data)
df=pd.DataFrame(data.data,columns=data.feature_names)
df['label']=data.target
df.columns=['sl','sw','pl','pw','label']

# X,y
X=np.array(df.iloc[:100,:2])
y=np.array(df.iloc[:100,-1])

y=np.array([-1 if i==0 else 1 for i in y  ])

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25)


0.96


In [29]:
class SVM():
    def __init__(self,max_iter=100,kernel='linear'):
        self.max_iter=max_iter
        self.kernel=kernel
        
    def _init_params(self,X_train,y_train):
        self.X=X_train
        self.Y=y_train
        self.b=0.0
        self.m=len(self.X)
        self.alphas=[1]*self.m
        self.C=1.0
        self.E=[self._E(i) for i in range(self.m)]
        
        
    def _choose_alpha(self):
        indexs=[ i for i in range(self.m) if 0<self.alphas[i]<self.C]
        no_kkt=[i for i in range(self.m) if i not in indexs]
        indexs.extend(no_kkt)
        for i in indexs:
            if self._kkt(i):
                continue
            if self.E[i]>=0:
                j=min(range(self.m),key=lambda a:self.E[a])
            else:
                j=max(range(self.m),key=lambda a:self.E[a])
            return i,j
    
    
    def _E(self,i):
        return self._g(i)-self.Y[i]
    
    def _kkt(self,i):
        yg=self.Y[i]*self._g(i)
        if self.alphas[i]==0:
            return yg>=1
        elif 0<self.alphas[i]<self.C:
            return yg==1
        else:
            return yg<=1
    
    def _g(self,j):
        return sum([self.alphas[i]*self.Y[i]*self._kernel(self.X[i],self.X[j]) for i in range(self.m)])+self.b
    
    def _kernel(self,x1,x2):
        if self.kernel=='linear':
            return np.dot(x1,x2)
        elif self.kernel=='poly':
            return (np.dot(x1,x2)+1)**2
        return 0

    
    def fit(self,X_train,y_train):
        self._init_params(X_train,y_train)
        for e in range(self.max_iter): 
            i,j=self._choose_alpha()
            
            if self.Y[i]==self.Y[j]:
                L=max(0,self.alphas[j]+self.alphas[i]-self.C)
                H=min(self.C,self.alphas[j]+self.alphas[i])
            else:
                L=max(0,self.alphas[j]-self.alphas[i])
                H=min(self.C,self.C+self.alphas[j]-self.alphas[i])
                
            eta=self._kernel(self.X[i],self.X[i])+self._kernel(self.X[j],self.X[j])-2*self._kernel(self.X[i],self.X[j])
            if eta<=0:
                continue
            alpha2_newunc=self.alphas[j]+self.Y[j]*(self.E[j]-self.E[i])/eta
            
            if alpha2_newunc>H:
                alpha2_new=H
            elif L<=alpha2_newunc<=H:
                alpha2_new=alpha2_newunc
            elif alpha2_newunc<L:
                alpha2_new=L
            
            alpha1_new=self.alphas[i]+self.Y[i]*self.Y[j]*(self.alphas[j]-alpha2_new)
            
            b1=-self.E[i]-self.Y[i]*self._kernel(self.X[i],self.X[i])*(alpha1_new-self.alphas[i])-\
            self.Y[j]*self._kernel(self.X[j],self.X[i])*(alpha2_new-self.alphas[j])+self.b
            
            b2=-self.E[j]-self.Y[i]*self._kernel(self.X[i],self.X[j])*(alpha1_new-self.alphas[i])-\
            self.Y[j]*self._kernel(self.X[j],self.X[j])*(alpha2_new-self.alphas[j])+self.b
            
            self.b=(b1+b2)/2
            self.alphas[i]=alpha1_new
            self.alphas[j]=alpha2_new
            
            self.E[i]=self._E(i)
            self.E[j]=self._E(j)
            
    def predict(self,xp):
        f=sum([self.alphas[i]*self.Y[i]*self._kernel(self.X[i],xp) for i in range(self.m)])+self.b
        if f>0:
            return 1
        else:
            return -1
        
    def score(self,X_test,y_test):
        acc=0
        for x,y in zip(X_test,y_test):
            if self.predict(x)==y:
                acc+=1
            
        return acc/len(y_test)
                

0.96


In [30]:
svm=SVM(max_iter=200)
svm.fit(X_train,y_train)
print(svm.score(X_test,y_test))

0.96


In [31]:

from sklearn.svm import LinearSVC
svc=LinearSVC()
svc.fit(X_train,y_train)
print(svc.score(X_test,y_test))

0.96
